In [31]:
!pip install -q crewai==0.55.0 langchain-mistralai neo4j==5.24.0  embedchain==0.1.120 langchain_anthropic

In [32]:
from crewai import Agent, Task, Crew, Process
from langchain_mistralai import ChatMistralAI
from neo4j import GraphDatabase
from langchain.agents import load_tools
from langchain.tools import tool
from langchain_anthropic import ChatAnthropic

In [33]:
llm = ChatAnthropic(model="claude-3-haiku-20240307", api_key="XXXXXXXXXXXXXXXX", temperature=0)

#llm = ChatMistralAI(model="mistral-large-latest", mistral_api_key="XXXXXXXXXXXXXX", temperature=0)

In [34]:
# Define the Neo4jConnectionTools class
class Neo4jConnectionTools:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def degree_centrality(self):
        with self.driver.session() as session:
            result = session.run("""
            CALL gds.degree.stream('myGraph')
            YIELD nodeId, score
            WHERE gds.util.asNode(nodeId).AgentId IS NOT NULL
            RETURN gds.util.asNode(nodeId).AgentId AS AgentId, score
            ORDER BY score DESC
            """)
            return [{"AgentId": record["AgentId"], "score": record["score"]} for record in result]

    def detect_client_communities(self):
        with self.driver.session() as session:
            result = session.run("""
            CALL gds.labelPropagation.stream('myGraph')
            YIELD nodeId, communityId
            RETURN communityId, collect(gds.util.asNode(nodeId).ClientId) AS Members
            ORDER BY size(Members) DESC
            """)
            return [{"communityId": record["communityId"], "Members": record["Members"]} for record in result]

    def detect_agent_communities(self):
        with self.driver.session() as session:
            result = session.run("""
            CALL gds.louvain.stream('myGraph')
            YIELD nodeId, communityId
            RETURN communityId, collect(gds.util.asNode(nodeId).AgentId) AS Members
            ORDER BY size(Members) DESC
            """)
            return [{"communityId": record["communityId"], "Members": record["Members"]} for record in result]

    def node_similarity(self):
        with self.driver.session() as session:
            result = session.run("""
            CALL gds.nodeSimilarity.stream('myGraph')
            YIELD node1, node2, similarity
            WHERE gds.util.asNode(node1).ClientId IS NOT NULL
            AND gds.util.asNode(node2).ClientId IS NOT NULL
            RETURN gds.util.asNode(node1).ClientId AS Node1, gds.util.asNode(node2).ClientId AS Node2, similarity
            ORDER BY similarity DESC
            LIMIT 10
            """)
            return [{"Node1": record["Node1"], "Node2": record["Node2"], "similarity": record["similarity"]} for record in result]

    def get_nodes(self, label):
        with self.driver.session() as session:
            result = session.run(f"MATCH (n:{label}) RETURN n")
            return [record["n"] for record in result]

# Initialize the Neo4j connection tool
neo4j_tool = Neo4jConnectionTools(uri="bolt://localhost:7687", user="neo4j", password="llmHackaton2024")


# Create wrapper functions for the tools
@tool('degree_centrality_tool')
def degree_centrality_tool():
    """degree_centrality_tool."""
    return neo4j_tool.degree_centrality()

@tool('detect_client_communities_tool')
def detect_client_communities_tool():
    """detect_client_communities_tool."""
    return neo4j_tool.detect_client_communities()

@tool('detect_agent_communities_tool')
def detect_agent_communities_tool():
    """detect_agent_communities_tool."""
    return neo4j_tool.detect_agent_communities()

@tool('node_similarity_tool')
def node_similarity_tool():
    """node_similarity_tool."""
    return neo4j_tool.node_similarity()


In [35]:
# Define the User Agent
user = Agent(
    role='User of an app called GuruWalk',
    goal=f'''Generate a user issue based on the app functionality, actions in the app, like booking, cancellations, report anomalies or any custom support issue, based only in this schema:

    Ticket Schema:

    Title: ticket_title
    Message: message
    Origin: origin
    NumberOfMessages: num_messages
    CreatedDate: created_date
    LastMessageDate: last_message_date
    Sender: sender_name
    Sender Mail: sender_email
    Status: status

    ''',
    verbose=True,
    backstory="""You are an avid traveler and a frequent user of the GuruWalk app, which you use to book guided walking tours in various cities around the world. You have a keen eye for detail and often find yourself providing feedback to improve the app's functionality. Whether it's booking a tour, canceling a reservation, reporting an issue, or seeking customer support, you are proactive in ensuring that your experiences are smooth and enjoyable. Your feedback is highly valued by the GuruWalk team, and you often help them identify and resolve issues quickly.""",
    llm=llm,
    function_calling_llm=llm
)

In [36]:
# Define the customer support agent
customer_support_agent = Agent(
    role='Customer Support Specialist',
    goal='Assist customers by resolving their inquiries efficiently, ensuring high satisfaction and retention and summary in a category the information of the ticket.',
    verbose=True,
    backstory="""With empathy and excellent communication skills, you excel at understanding customer needs,
    addressing issues promptly, and providing accurate information to create a positive customer experience.""",
    allow_delegation=True,  # Typically, a support agent handles issues directly unless escalating to another team.
    max_iter=1,  # Allow more iterations for interactive troubleshooting.
    max_rpm=15,  # Adjusted RPM based on a real-time response scenario.
    llm=llm,  # Large Language Model for natural interactions.
    function_calling_llm=llm  # For enhanced functionalities like escalating or retrieving FAQs.
)

In [37]:
# Senior Machine learning engineer
ml_engineer = Agent(
    role='Senior Machine learning engineer',
    goal=f'Create AI agents with a single task to resolve a ticket bases on custumers input,  following the schema: [role, goal, backstory]',
    verbose=True,
    backstory="""You are an expert in Multiagentic systems design and using Crewai sdk with python yo are able to generate the best Agent for perform the action and solve the custumer issue""",
    llm=llm,
    function_calling_llm=llm
)


In [38]:
# Senior Knowledge Engineer
kg_engineer = Agent(
    role='Senior Knowledge Engineer',
    goal=f'Execute the Neo4j queries and funtions for Graph Machine Learning,  following the schema: [role, goal, backstory]',
    verbose=True,
    backstory="""You are an expert in Cypher query language for Graph Databases using Neo4j and Knowledge's Graphs""",
    llm=llm,
    function_calling_llm=llm
)

In [39]:
# Define the asynchronous tasks
generate_ticket = Task(
    description='''
    Generate a user issue Ticket using this Schema:

        Title: ticket_title
        Message: message
        Origin: origin
        NumberOfMessages: num_messages
        CreatedDate: created_date
        LastMessageDate: last_message_date
        Sender: sender_name
        Sender Mail: sender_email
        Status: status

    ''',
    expected_output="The Schema provided filled with a very realistic custumer issue",
    tools=[],
    agent=user,
    async_execution=False
)

# Define the task
summary_ticket_task = Task(
    description=(
        "Analyze the customer ticket and summarize the information into relevant categories. "
        "The agent should identify the main issue, categorize it appropriately, and provide a detailed description "
        "for each possible category in the customer support industry. This will help in generating a solution for the customer ticket."
    ),
    expected_output="A detailed description of the possible categories in the Customer Support industry to classify the issues and generate a solution for the customer ticket.",
    agent=customer_support_agent,
    tools=[],  # Add any tools if necessary
    async_execution=False
)

# Define the task
create_ai_agent_task = Task(
    description=(
        "Develop an AI agent that can resolve a customer ticket based on the provided input. "
        "The agent should analyze the customer's issue, determine the best course of action, and provide a solution. "
        "Ensure the agent follows the schema: [role: string, goal: string, backstory: string]."
    ),
    expected_output="based in the Return from Knowledge Graph from the 3 best Agents to solve the customer issue, return just a JSON for instanciate a Crewai Crew of Agents including its role, goal, and backstory.",
    agent=ml_engineer,
    tools=[],  # Add any tools if necessary
    async_execution=False
)

# Define the task
execute_neo4j_queries_task = Task(
    description=(
        "Execute Neo4j queries and functions for Graph Machine Learning based on the provided requirements. "
        "The agent should utilize its expertise in Cypher query language and Neo4j to perform the necessary operations. "
        "Ensure the agent follows the schema: [role, goal, backstory]."
    ),
    expected_output="Return in a JSON the resutls from executed the tool.",
    agent=kg_engineer,
    tools=[],  # Add any tools if necessary
    async_execution=True
)

In [40]:
# Forming the crew with a hierarchical process including the manager
crew = Crew(
    manager_llm=llm,
    agents=[user, customer_support_agent, kg_engineer, ml_engineer],
    tasks=[generate_ticket, summary_ticket_task, execute_neo4j_queries_task, create_ai_agent_task],
    process=Process.sequential,
    verbose=True
)

In [41]:
try:
    # Kick off the crew's work asynchronously
    results = crew.kickoff()
    # Display results if successful
except Exception as e:
    # Handle and display the error
    print(f"An error occurred: {str(e)}")


 [2024-10-27 07:15:20][DEBUG]: == Working Agent: User of an app called GuruWalk
 [2024-10-27 07:15:20][INFO]: == Starting Task: 
    Generate a user issue Ticket using this Schema:

        Title: ticket_title
        Message: message
        Origin: origin
        NumberOfMessages: num_messages
        CreatedDate: created_date
        LastMessageDate: last_message_date
        Sender: sender_name 
        Sender Mail: sender_email
        Status: status
    
    


> Entering new CrewAgentExecutor chain...
Thought: To generate a realistic user issue ticket, I will need to gather more information about the specific problem the user is facing with the GuruWalk app. I should ask the user for more details about the issue they encountered.

Action: Ask question to coworker
Action Input: {
  "question": "Can you please provide more details about the issue you encountered with the GuruWalk app?",
  "context": "I'm trying to generate a realistic user issue ticket based on the schema provided

In [42]:
print("Crew Work Results:")
print(results)

Crew Work Results:
{
  "agents": [
    {
      "role": "Customer Support Specialist",
      "goal": "Resolve the customer's issue with the Widget Pro product",
      "backstory": "The Customer Support Specialist has been working in customer service for 5 years and is known for their patience and problem-solving skills. They are responsible for investigating customer issues and determining the best course of action to resolve them."
    },
    {
      "role": "Senior Knowledge Engineer",
      "goal": "Leverage graph database technologies and knowledge graphs to identify the best solution for the customer's problem with the Widget Pro product",
      "backstory": "The Senior Knowledge Engineer has extensive expertise in using Cypher query language for Graph Databases, Neo4j, and Knowledge Graphs. They are responsible for diagnosing issues, analyzing data, and providing comprehensive solutions to customers."
    },
    {
      "role": "Product Specialist",
      "goal": "Provide technica